# train the model for QA

* was previously trained on our MLM data
* now traing for QA task on standard SQUAD

In [1]:
cd ../..

/home/sambeck/code/nlpfp


In [2]:
import datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, \
    AutoModelForQuestionAnswering, Trainer, TrainingArguments, HfArgumentParser, pipeline
from helpers import prepare_dataset_nli, prepare_train_dataset_qa, \
    prepare_validation_dataset_qa, QuestionAnsweringTrainer, compute_accuracy
import os
import json

NUM_PREPROCESSING_WORKERS = 2

In [3]:
cd experiments/mlm

/home/sambeck/code/nlpfp/experiments/mlm


In [24]:
MODEL='./model/'
TASK='qa'
DATASET='squad'
MAX_LENGTH=128
TRAIN_MAX_SAMPLES=5000

### compare new Checklist-based dataset with Squad

(you can make a basic Checklist dataset using the make_checklist_dataset notebook)

In [6]:
sqd = datasets.load_dataset('squad')

Reusing dataset squad (/home/sambeck/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


In [14]:
dataset = sqd

### Model

In [16]:
model_class = AutoModelForQuestionAnswering
# Initialize the model and tokenizer from the specified pretrained model/checkpoint
model = model_class.from_pretrained(MODEL)
tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=True)

Some weights of the model checkpoint at ./model/ were not used when initializing ElectraForQuestionAnswering: ['generator_lm_head.bias', 'generator_predictions.dense.weight', 'generator_predictions.LayerNorm.weight', 'generator_lm_head.weight', 'generator_predictions.LayerNorm.bias', 'generator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at ./model/ and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably

In [17]:
# model pipeline (includes tokenization)
mp = pipeline("question-answering", tokenizer=tokenizer, model=model, device=0)

In [18]:
pred = mp(question="Which color is the dog?", context="There is a black dog.", truncation=True, )
print(pred)

{'score': 0.055828630924224854, 'start': 11, 'end': 21, 'answer': 'black dog.'}


### Set up dataset for training

In [25]:
train_dataset_featurized = None
eval_dataset_featurized = None
train_dataset = dataset['train']
eval_dataset = dataset['validation']

In [26]:
prepare_train_dataset = lambda exs: prepare_train_dataset_qa(exs, tokenizer)
prepare_eval_dataset = lambda exs: prepare_validation_dataset_qa(exs, tokenizer)

if TRAIN_MAX_SAMPLES:
    train_dataset = train_dataset.select(range(TRAIN_MAX_SAMPLES))

print('featurize train...')
train_dataset_featurized = train_dataset.map(
    prepare_train_dataset,
    batched=True,
    num_proc=NUM_PREPROCESSING_WORKERS,
    remove_columns=train_dataset.column_names
)

print('featurize test...')
eval_dataset_featurized = eval_dataset.map(
    prepare_eval_dataset,
    batched=True,
    num_proc=NUM_PREPROCESSING_WORKERS,
    remove_columns=eval_dataset.column_names
)


featurize train...
featurize test...


Loading cached processed dataset at /home/sambeck/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-647933d23d01e8b5.arrow
Loading cached processed dataset at /home/sambeck/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453/cache-c6a30d2b93d27863.arrow


In [27]:
trainer_class = Trainer
eval_kwargs = {}
# If you want to use custom metrics, you should define your own "compute_metrics" function.
# For an example of a valid compute_metrics function, see compute_accuracy in helpers.py.
compute_metrics = None
# For QA, we need to use a tweaked version of the Trainer (defined in helpers.py)
# to enable the question-answering specific evaluation metrics
trainer_class = QuestionAnsweringTrainer
eval_kwargs['eval_examples'] = eval_dataset
metric = datasets.load_metric('squad')
compute_metrics = lambda eval_preds: metric.compute(
    predictions=eval_preds.predictions, references=eval_preds.label_ids)

In [28]:
# This function wraps the compute_metrics function, storing the model's predictions
# so that they can be dumped along with the computed metrics
eval_predictions = None
def compute_metrics_and_store_predictions(eval_preds):
    global eval_predictions
    eval_predictions = eval_preds
    return compute_metrics(eval_preds)

### Train

In [29]:
# Initialize the Trainer object with the specified arguments and the model and dataset we loaded above
trainer = trainer_class(
    model=model,
    train_dataset=train_dataset_featurized,
    eval_dataset=eval_dataset_featurized,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_and_store_predictions
)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [31]:
trainer.train()


***** Running training *****
  Num examples = 5009
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1881


Step,Training Loss
500,3.150800
1000,1.914000
1500,1.534900


Saving model checkpoint to tmp_trainer/checkpoint-500
Configuration saved in tmp_trainer/checkpoint-500/config.json
Model weights saved in tmp_trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in tmp_trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in tmp_trainer/checkpoint-500/special_tokens_map.json
Saving model checkpoint to tmp_trainer/checkpoint-1000
Configuration saved in tmp_trainer/checkpoint-1000/config.json
Model weights saved in tmp_trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in tmp_trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in tmp_trainer/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to tmp_trainer/checkpoint-1500
Configuration saved in tmp_trainer/checkpoint-1500/config.json
Model weights saved in tmp_trainer/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in tmp_trainer/checkpoint-1500/tokenizer_config.json
Special tokens file saved in tmp_traine

TrainOutput(global_step=1881, training_loss=2.025986293729856, metrics={'train_runtime': 556.8469, 'train_samples_per_second': 26.986, 'train_steps_per_second': 3.378, 'total_flos': 439051790370816.0, 'train_loss': 2.025986293729856, 'epoch': 3.0})

In [32]:
pred = mp(question="Which color is the dog?", context="There is a black dog.", truncation=True, )
print(pred)

{'score': 0.9625220894813538, 'start': 11, 'end': 16, 'answer': 'black'}


In [33]:
pred = mp(question='Who is the most awesome?', context='William is awesome, but John is more awesome', truncation=True, )
print(pred)

{'score': 0.3188024163246155, 'start': 24, 'end': 28, 'answer': 'John'}


In [34]:
pred = mp(question="Which thing is hot?", context="There is a cold gopher, a polar bear, and a hot snake.", truncation=True, )
print(pred)

{'score': 0.12154275923967361, 'start': 26, 'end': 53, 'answer': 'polar bear, and a hot snake'}


In [35]:
pred = mp(question="Which thing is least hot?", context="There is a cold gopher, a polar bear, and a hot snake.", truncation=True, )
print(pred)

{'score': 0.2318418323993683, 'start': 16, 'end': 36, 'answer': 'gopher, a polar bear'}


### Save

In [36]:
trainer.save_model('./model-qa-trained')

Saving model checkpoint to ./model-qa-trained
Configuration saved in ./model-qa-trained/config.json
Model weights saved in ./model-qa-trained/pytorch_model.bin
tokenizer config file saved in ./model-qa-trained/tokenizer_config.json
Special tokens file saved in ./model-qa-trained/special_tokens_map.json
